# Inference

In [ ]:
########################## CONDA INSTALLATION ##########################
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x ./Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!rm ./Miniconda3-latest-Linux-x86_64.sh

In [ ]:
##################### CLONE THE REPOSITORY #######################
!git clone https://github.com/divyjx/Video-LLaVA
%cd Video-LLaVA

Cloning into 'Video-LLaVA'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 306 (delta 44), reused 50 (delta 17), pack-reused 228
Receiving objects: 100% (306/306), 69.77 MiB | 35.04 MiB/s, done.
Resolving deltas: 100% (77/77), done.
/content/Video-LLaVA


In [ ]:
########################## CONDA ENVIRONMENT CREATION ##########################
%%shell
eval "$(conda shell.bash hook)"
conda create -n videollava python=3.10 -y
conda init --all

In [ ]:
############################# DOWNLOAD REQUIRED LIBRARIES ##############################
%%shell
eval "$(conda shell.bash hook)"
conda activate videollava

pip install --upgrade pip  # enable PEP 660 support
pip install -e .
pip install -e ".[train]"
pip install flash-attn --no-build-isolation
pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d
pip install tensorboardX
pip install pycocoevalcap

In [ ]:
######################### DOWNLOAD MODEL ##########################
from huggingface_hub import snapshot_download

# no trianing
# snapshot_download(repo_id= "LanguageBind/Video-LLaVA-7B", local_dir = 'checkpoints/Video-LLaVA-7B', local_dir_use_symlinks=True)

# trained on 10k samples
# snapshot_download(repo_id= "divyjx/VideoLLaVA", local_dir = 'checkpoints/Video-LLaVA-7B', local_dir_use_symlinks=True)

# trained on 200k samples (final model)
snapshot_download(repo_id= "divyjx/VideoLLaVA10k", local_dir = 'checkpoints/Video-LLaVA-7B', local_dir_use_symlinks=True)


In [ ]:
######################### DOWNLOAD DATASET #########################
from huggingface_hub import snapshot_download
snapshot_download(repo_id= "divyjx/VideoLLaVA_dataset", local_dir = 'tuning_data', local_dir_use_symlinks=True, repo_type = 'dataset')

# Batch Inference

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate videollava
python inference.py --model-path "checkpoints/Video-LLaVA-7B" --instruct-path "tuning_data/content_10.json" --output-file-path "output.json" --video-folder "videos" --image-folder "images"

# Evaluation

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate videollava
#######################################################
python eval.py --prediction-file-path "output.json" --eval-file-path "eval.json" --input-file-path "tuning_data/content_10.json" --type-of-data "content"